### <b>Fastapi</b>
- Fastapi is your ASGI(Asynchronous server gateway interface) supported framework
- Asynchronous programming (async/await) to handle many concurrent requests efficiently.
- Automatic API documentation (SWAGGER http://host:port/docs)
- Pydantic library to automatically validate request and response datas

### <b>Uvicorn</b>
- Uvicorn is the server that acts as a bridge between your fastapi application and the browser
- Uvicorn reads the address(host) door(port) from browser request and delivers the request if person(app) is active(running: uvicorn main:app)
- Browser sends bytes → Uvicorn translates bytes to ASGI → ASGI call is made → FastAPI executes logic → Uvicorn converts ASGI to bytes -> sends bytes back

### <b>Python-dotenv</b>
- load_dotenv() injects values into the global OS environment (os.environ) by reading .env file
- dotenv_values() reads .env file and returns a local dictionary with values as key-value pair. When explicit configuration matters
- load_dotenv(find_dotenv()) - Walks up directories, Finds nearest .env

### <b>SQLAlchemy & Drivers (asyncpg, psycopg2 ...)</b>

#### SQLAlchemy

- SQLAlchemy is a package we used as the main center to connect to databases like postgres, mysql databases etc
- SQLAlchemy handles all Connection pooling, Transaction isolation, Concurrency safety, has ORM, used for relational databases.. and many more
- URL -> SQLAlchemy -> Engine -> Pool allocation -> Request arrival -> Engine -> Session -> Database -> Engine -> Close session
- SQLAlchemy does NOT open a database connection when you create the engine. It opens a connection only when the first query actually needs it.
    - Why lazy connections are important ??

        If SQLAlchemy were eager instead of lazy:
        
        - App startup would:
            - Open DB connections immediately
            - Fail if DB is temporarily down
        - You’d waste connections even if:
            - App never hits DB
            - Some routes don’t need DB
- Connections are only created when first query runs, when connections is required.
    - when there is session.execute(query), it says “I actually need a connection → open one now.”
- It will be empty pools if there is no requirements, Engine say “I know how to connect, but I haven’t connected yet.”

#### Driver

- Although we don't manually import drivers anywhere in code, we must need driver to connect to the appropriate databases
    - PostgreSQL (Async) - asyncpg - postgresql+asyncpg://
    - PostgreSQL (Sync) - psycopg2 - postgresql+psycopg2://
    - MySQL (Async) - aiomysql - mysql+aiomysql://
    - MySQL (Sync) - pymysql - mysql+pymysql://
- Its passed along with url, Engine decodes the driver

#### Engine
- Guard of the connection pool
- Manages the connetion pool
- Assigns tokens (session) to new requests.
- if 10 connections are there, each parallel reqeusts are passed to different connections hence different session. 11th parallel request has to wait to get a session until one of them releases their session.
- create engine globally instead of creating in each request inside our application
- Engine creation is expensive, behind the scenes:
    - DNS resolution - parsing the url
    - TCP, TLS handshake
    - Authentication - username password verification etc.
    - session initilization
    - kernel resources
- Engine creation might take around 10-100ms per connection, that X100 request = disaster. so create engine(guard) globally

#### Session

- It begins when a connection is established and ends upon disconnection, log out, or a timeout due to inactivity. 
- How session is created:
    - when you call a request:
        ```bash
        @app.get("/health/db")
        async def db_health_check(db: AsyncSession = Depends(get_db)):
            ...
        ```
    - It calls:
        ``` bash
        async def get_db():
            async with AsyncSessionLocal() as session:
                    yield session
        ```
    - Under the hood (simplified):
        ``` bash
        gen = get_db()
        session = await gen.__anext__()   # before yield
        #route executes here
        await gen.aclose()                # after response
        ```

#### Work flow:

- App starts
- Url read by engine in SQLAlchemy
- <b>Engine</b> created (guard hired)
- Pool initialized (tunnels mapped, not opened yet)
- Request arrives
- <b>Session</b> created (pass issued)
- Connection borrowed (tunnel opened or reused) (managed by Engine(guard))
- Query executed
- Request finishes
- Session closes
- Connection returned to pool
- Engine notes that this connection is free and can be used by others
- App shuts down
- Engine disposed
- All tunnels closed (Guard goes home)

#### Mental models:

- Engine = connection manager
- Pool = performance optimizer
- Session = request conversation
- Async = concurrency without blocking
- Multiple users ≠ multiple engines
- DB guarantees data safety, not SQLAlchemy

### <b>Alembic and sqlalchemy.orm</b>

#### ORM
- ```ORM``` - object relational mapper, its a tool, it maps between two worlds(database and class)
- used to map the object in python to tables in database
- we connect rows in database to objects in python
- reads rows in database table as corresponding orm model defined in python
- Define the ORM model in ```model.py``` files

    ##### DeclaraiveBase

    - we declare a class defined as orm model when we pass ```DeclarativeBase``` in that class argument
    - once DeclarativeBase is defined, it will store the metadata of the classes that inherits from DeclarativeBase
    - it will store table name, columns, column features etc in ```DecalarativeBase.metadata```
    - we use ```DeclarativeBase.metadata.create_all()``` method to programmatically create tables in database.
    - it is not migration-aware and does not track schema changes

    ##### Mapped and mapped_column()
    - ```Mapped``` and ```mapped_column()``` is a new feature to identify the type of attribute for python.
    - old approach we dont know the type of attribute till runtime. but typing method will initilize the type earlier than runtime
    - old style:
        - ```bash 
            id = Column(Integer, primary_key=True) 
            ```
        - From Python’s perspective: 
            ```bash 
            type(User.id) == # Column
            ```
        - But at runtime: 
            ```bash 
            user = User()
            type(user.id)  # int
            ```
        - Without Mapped:
            - IDE cannot help you
            - Type errors show up only at runtime
    - New style:
        - ```bash 
            id: Mapped[int] = mapped_column(primary_key=True) 
            ```
        - Mapped[int] tells Python “This attribute behaves like int when accessed”
        - mapped_column() tells SQLAlchemy primary key, nullable, index, constraints. 
        - Column type will be infered from Mapped[int]
        - with Mapped:
            - Type errors caught early
            - Large-team friendly

#### Alembic
- Use Alembic for migrating database schema and tracking schema versions
- alembic will store the changes in its file like git does.
- you can think alembic as git for databases.
- always keep database updations and creation seperate from app struture
- Schema creation and updates are kept separate so that application runtime remains safe and free from schema mutation logic
    
    What happens if we keep schema mutation logic in app, eg. on_startup?
    - App v1 runs → creates tables
    - You deploy v2 (new column added)
    - App starts again → create_all() does NOTHING
    - DB schema ≠ code schema
    - App crashes later

- manual intervenion of changing the names of tables in python or ORM model might cause issue while alembic migration and it might give wrong migration script
- always cross check migration script before migrating to actualy db
- in migration sript:
    - upgrade() → Applies the migration (creates tables, columns, indexes, etc.) when you run alembic upgrade head or alembic upgrade +1.
    - downgrade() → Reverses the migration. It's used when you run alembic downgrade -1 or alembic downgrade <previous_revision>.
    - for index operations, alembic follow a naming convention: ix_<schema>_<tablename>_<columnnames>, The name is long but valid
